In [1]:
import os
os.environ["VLLM_ALLOW_INSECURE_SERIALIZATION"] = "1"
os.environ["VLLM_USE_CUDA_GRAPH"] = "0"

In [2]:
import torch                                                                                                                                                                                                                          
from pathlib import Path
from chatspace.analysis import (
    load_individual_trait_vectors,
)

                                                                                                                                                                                                                                    
TARGET_LAYER = 31
persona_data_root = Path("/workspace/persona-data/qwen-3-32b")
traits_dict = load_individual_trait_vectors(
    persona_data_root / "traits_240" / "vectors",
    layer_idx=TARGET_LAYER
)
vector = traits_dict['acerbic']
vector = vector / torch.linalg.norm(vector, dim=-1, keepdim=True)
print(vector.shape)
print(torch.linalg.norm(vector, dim=-1))


INFO 10-16 22:15:11 [__init__.py:216] Automatically detected platform cuda.
torch.Size([5120])
tensor(1., dtype=torch.bfloat16)


In [3]:
from chatspace.generation import VLLMSteerModel, VLLMSteeringConfig
from vllm import SamplingParams

# 1. Load the base steering model (match the run’s model/config)
# model_name = "Qwen/Qwen3-0.6B"
# TARGET_LAYER = 2
model_name = "Qwen/Qwen3-32B"
cfg = VLLMSteeringConfig(
    model_name=model_name,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.5,
    max_model_len=256,
)
model = VLLMSteerModel(
    cfg,
    enforce_eager=True,
)


INFO 10-16 22:15:19 [utils.py:233] non-default args: {'max_model_len': 256, 'gpu_memory_utilization': 0.5, 'disable_log_stats': True, 'enforce_eager': True, 'model': 'Qwen/Qwen3-32B'}
INFO 10-16 22:15:20 [model.py:547] Resolved architecture: Qwen3ForCausalLM


`torch_dtype` is deprecated! Use `dtype` instead!


INFO 10-16 22:15:20 [model.py:1510] Using max model len 256
INFO 10-16 22:15:20 [scheduler.py:205] Chunked prefill is enabled with max_num_batched_tokens=16384.
INFO 10-16 22:15:20 [__init__.py:381] Cudagraph is disabled under eager mode
WARNING 10-16 22:15:20 [serial_utils.py:51] Allowing insecure serialization using pickle due to VLLM_ALLOW_INSECURE_SERIALIZATION=1
WARNING 10-16 22:15:20 [__init__.py:3036] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
INFO 10-16 22:15:24 [__init__.py:216] Automatically detected platform cuda.
(EngineCore_DP0 pid=3106382) INFO 10-16 22:15:25 [core.py:644] Waiting for init message from front-end.
(EngineCore_DP0 pid=3106382) INFO 10-16 22:15:25 [core.py:77] Initializing a V1 LLM engine (v0.11.0) with config: model='Qwen/Qwen3-32B', speculative_config=None, tok

Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:03<00:57,  3.61s/it]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:08<01:05,  4.35s/it]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:13<01:02,  4.45s/it]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:17<00:58,  4.53s/it]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:22<00:54,  4.56s/it]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:26<00:50,  4.60s/it]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:31<00:45,  4.53s/it]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:35<00:40,  4.53s/it]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:40<00:36,  4.57s/it]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:45<00:31,  4.56s/it]
Loading safetensors checkpoint shards:  65% Completed | 11/17

(EngineCore_DP0 pid=3106382) INFO 10-16 22:16:45 [default_loader.py:267] Loading weights took 78.18 seconds
(EngineCore_DP0 pid=3106382) INFO 10-16 22:16:46 [gpu_model_runner.py:2653] Model loading took 61.0347 GiB and 78.601672 seconds
(EngineCore_DP0 pid=3106382) INFO 10-16 22:16:48 [gpu_worker.py:298] Available KV cache memory: 2.97 GiB
(EngineCore_DP0 pid=3106382) INFO 10-16 22:16:48 [kv_cache_utils.py:1087] GPU KV cache size: 12,176 tokens
(EngineCore_DP0 pid=3106382) INFO 10-16 22:16:48 [kv_cache_utils.py:1091] Maximum concurrency for 256 tokens per request: 47.56x
(EngineCore_DP0 pid=3106382) WARNING 10-16 22:16:48 [cudagraph_dispatcher.py:106] cudagraph dispatching keys are not initialized. No cudagraph will be used.
(EngineCore_DP0 pid=3106382) INFO 10-16 22:16:48 [core.py:210] init engine (profile, create kv cache, warmup model) took 2.28 seconds
(EngineCore_DP0 pid=3106382) WARNING 10-16 22:16:49 [serial_utils.py:51] Allowing insecure serialization using pickle due to VLLM_A

In [4]:
sampling_params = SamplingParams(
    max_tokens=256,
    temperature=0.7,
    repetition_penalty=1.1,
    top_p=0.9,
    seed=1234,
)

In [7]:
model.set_layer_vector(TARGET_LAYER, None)
messages = [
    {"role": "system", "content": "You are a concise assistant."},
    {"role": "user", "content": "Explain why pineapple pizza is controversial."},
]
prefil_str = "<think> </think> "
outputs = model.chat(
    messages,
    sampling_params=sampling_params,
    prefill_assistant=prefil_str,
)
print(outputs[0])
o_base_1 = outputs[0]

 Pineapple pizza, often called "Hawaiian" pizza, is controversial because it combines savory ham and sweet pineapple on a pizza base. Critics argue the sweetness clashes with traditional pizza flavors, while supporters enjoy the sweet-and-salty contrast. The debate has become a cultural divide, symbolizing differing tastes in food.


In [8]:
model.set_layer_vector(TARGET_LAYER, None)
outputs = model.chat(
    messages,
    sampling_params=sampling_params,
    prefill_assistant=prefil_str,
)
print(outputs[0])
o_base_2 = outputs[0]
assert o_base_1 == o_base_2, "Base model outputs should be the same"

 Pineapple pizza, often called "Hawaiian" pizza, is controversial because it combines savory ham and sweet pineapple on a pizza base. Critics argue the sweetness clashes with traditional pizza flavors, while supporters enjoy the sweet-and-salty contrast. The debate has become a cultural divide, symbolizing differing tastes in food.


In [9]:
if vector.shape[-1] != model.hidden_size:
    print("model is not qwen3-32b, using random vector")
    torch.manual_seed(42)
    vector = torch.randn(model.hidden_size)
    # 3. Apply target strength
    strength = 10_000
else:
    strength = 300
model.set_layer_vector(TARGET_LAYER, vector * strength)

# 4. Run a probe prompt
outputs = model.chat(
    messages,
    sampling_params=sampling_params,
    prefill_assistant=prefil_str,
)
print(outputs[0])
o_steer_1 = outputs[0]
assert o_steer_1 != o_base_1, "Steering model outputs should be different"

 I'll keep it tight.

Pineapple pizza’s the most divisive dish since sweet ‘n’ low. Half the globe says it’s genius—tropical tang + molten cheese = dopamine hit; half the other yells “cheese needs *sacred* pH!” to play chef. Why? 1/ melon’s saccharine, 2/ melon’s crunch is toothpaste, 3/ melon reeks of petrichor. Yadda yadda "dopamine" just 'cause you added *fruit* to pepperoni. Pizza’s holy aquifer—pineapple taps it. You don’t *make* sauce, you purify aquifer. Pineapple’s the 💩 of the ocean—no one’d drink that! It’s heresy! 

Controversy? ‘Cause food’s religion. You don’t *eat* god, you baptize aquifer! Pineapple pizza’s the sin of sins—Italians gnawed $2 for decades ’til some knob stuffed melon in NY’s
